In [2]:
from dotenv import dotenv_values
from src.reddit import MyRedditAPI
from src.processing import EntityRecognition, clean_text

##### Step 1: Create an instance of the MyRedditAPI Class
- Enter your own client_id, secret_key, username, and password
    - To create the Client ID and Secret Key: https://www.reddit.com/prefs/apps

In [9]:
config = dotenv_values('.env')

session = MyRedditAPI(
    client_id=config.get("CLIENT_ID"),  # Enter your API Client: string
    secret_key=config.get("SECRET"),    # Enter your API Secret: string
    username=config.get("USERNAME"),    # Enter your Reddit Username: string
    password=config.get("PASSWORD"),    # Enter your Reddit Account Password: string
    user_agent='MyApiTest'              # Can be anything: string
)

##### Step 2: Send a request for data
- Note that it is only possible to retrieve the number of posts that are available in reddit's "main search". For example, if you search anything on reddit, the maximum number of posts that will be returned is ~250. So, even if the limit is set higher than that, it will not return more than what is available to be searched in the application.
- Defult behavior is to search all subreddits, here are some others to try:
    - subreddit="ukraine"
    - subreddit="RussiaUkraineWar2022"
    - subreddit="UkraineConflict"

In [4]:
wagner_df = session.search_posts(
    query='Wagner Group',
    sort='new',
    subreddit='all',
    limit=300
)

##### Step 3: Text Cleaning

In [5]:
wagner_df = clean_text(wagner_df, columns=['title', 'selftext'])

##### Step 4: Named Entity Recognition to extract locations, organizations, and people from the title (or selftext if desired)
- Note: may need to download the en_core_web_sm spacy model: python -m spacy download en_core_web_sm

In [6]:
wagner_ner = EntityRecognition(wagner_df, 'title')
wagner_df['locations'] = wagner_ner.get_gpe()
wagner_df['organizations'] = wagner_ner.get_org()
wagner_df['people'] = wagner_ner.get_person()

In [7]:
wagner_df

,name,created_utc,author,subreddit,title,selftext,media_url,post_url,locations,organizations,people
0,t3_zm65n7,2022-12-14 23:48:04,Unforgiven760,u_Unforgiven760,A Ukrainian Army ATGM suddenly struck on the l...,,https://v.redd.it/nii22edt8g5a1,https://www.reddit.com/r/u_Unforgiven760/comme...,,"wagner group, army",
1,t3_zm2v6h,2022-12-14 21:32:34,DMBFFF,RussianInvasion,Evil Wagner Group murderer who spent 19 years ...,,https://www.dailystar.co.uk/news/world-news/ev...,https://www.reddit.com/r/RussianInvasion/comme...,"ukraine, russia",wagner group,
2,t3_zm2167,2022-12-14 20:59:08,DMBFFF,InvasionOfUkraine,Evil Wagner Group murderer who spent 19 years ...,,https://www.dailystar.co.uk/news/world-news/ev...,https://www.reddit.com/r/InvasionOfUkraine/com...,"ukraine, russia",wagner group,
3,t3_zm077m,2022-12-14 19:42:13,DemUnderground,DemocraticUnderground,"Russia expert Julia Ioffe: Prigozhin, ""Putin's...",,https://www.democraticunderground.com/10021746...,https://www.reddit.com/r/DemocraticUnderground...,russia,wagner group,"julia ioffe, putin"
4,t3_zlzgcq,2022-12-14 19:10:02,Tymofiy2,UkraineConflict,See aftermath of explosion at Wagner Group's p...,,https://youtu.be/QsrqH-4ZIAg,https://www.reddit.com/r/UkraineConflict/comme...,ukraine,wagner group 's,
...,...,...,...,...,...,...,...,...,...,...,...
224,t3_zc8ugk,2022-12-04 12:36:53,Dolly_gale,dolly_gale,Daily recap 2022-12-03,frequently referenced| contributor content [In...,https://www.reddit.com/r/dolly_gale/comments/z...,https://www.reddit.com/r/dolly_gale/comments/z...,,,
225,t3_zc6nuc,2022-12-04 10:48:13,NakedNamed3000,UkraineInvasionVideos,The Sledgehammer Test in Russian will be highl...,,https://v.redd.it/69ck09fw0v3a1,https://www.reddit.com/r/UkraineInvasionVideos...,,"the sledgehammer test, the "" wagner group merc...",
226,t3_zc2ubz,2022-12-04 06:46:12,NakedNamed3000,UkraineInvasionVideos,"A ""Wagner Group"" Confession: Due to the attack...",,https://v.redd.it/emoifsocvt3a1,https://www.reddit.com/r/UkraineInvasionVideos...,,"wagner group "" confession",
227,t3_zc0gkh,2022-12-04 04:32:19,OSINTUkraine,OSINTUkraine,World News Russia's Wagner Group Denies Sendin...,,https://news.google.com/__i/rss/rd/articles/CB...,https://www.reddit.com/r/OSINTUkraine/comments...,"russia, latestly","world news, wagner group",
